In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
import os
import numpy as np
import librosa
import sys
import tensorflow as tf
from keras.models import load_model

target_length = 3341

In [3]:
os.chdir('/content/drive/MyDrive/Projects/AlarmWaterClassification')

In [8]:
def convert_keras2qttflite(feature_folder, keras_model_file, tflite_model_file):
    def representative_dataset_test():
        list_np = glob.glob(f"{feature_folder}/**/*.npy", recursive=True)
        for np_f in list_np:
            np_data = np.load(np_f)
            np_data = np.array(np_data, dtype=np.float32, ndmin=2)
            yield [np.expand_dims(np_data, axis=0)]

    keras_model = load_model(keras_model_file)
    # Convert the model to the TensorFlow Lite format with quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_test
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8  # tf.lite.constants.INT8
    converter.inference_output_type = tf.uint8

    tflite_model = converter.convert()

    open(tflite_model_file, "wb").write(tflite_model)


def pad_features(features, target_length):
    if len(features) < target_length:
        padded_features = np.zeros((target_length, 1))
        padded_features[:len(features), 0] = features
    else:
        padded_features = features[:target_length]
        padded_features = padded_features.reshape(-1, 1)  # Ensure it has the shape (timesteps, channels)
    return padded_features

def get_STFT(file):
    # read audio data
    audio_data, sample_rate = librosa.load(file, sr=16000)

    # noise reduction
    noisy_part = audio_data[0:25000]
    #     reduced_noise = nr.reduce_noise(audio_clip=audio_data, noise_clip=noisy_part, verbose=False)
    # reduced_noise = nr.reduce_noise(y=audio_data, y_noise=noisy_part, sr=sample_rate)
    reduced_noise = audio_data

    # trimming
    trimmed, index = librosa.effects.trim(reduced_noise, top_db=20, frame_length=512, hop_length=64)

    # extract features
    stft = np.abs(librosa.stft(trimmed, n_fft=512, hop_length=256, win_length=512))

    return stft


def compare_model_kerasandtflite(audio_file, keras_model_file, tflite_model_file, trim_flag=True, same_training=False):
    feature = get_STFT(audio_file)
    feature = feature
    # print('Original feature vector', np.shape(feature))
    model_keras = load_model(keras_model_file)
    step = 10
    # print('After Split ....')
    for i in range(0, feature.shape[1], step):
        new_feature = feature[:, i:i + step]

        print('Stft: ', np.shape(new_feature))

        mfccs = librosa.feature.mfcc(y=new_feature, sr=16000, n_mfcc=13, n_fft=512)
        print(f"mfcc before mean: {np.shape(mfccs)}")

        mfccs = np.mean(mfccs.T, axis=0)
        print(f"mfcc after mean: {np.shape(mfccs)}")

        mfccs = mfccs.flatten()

        pad = pad_features(mfccs, target_length)

        new_feature = pad

        pred = model_keras.predict(np.asarray([new_feature]), verbose=0)

        interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
        interpreter.allocate_tensors()

        input_details = interpreter.get_input_details()[0]
        output_details = interpreter.get_output_details()[0]

        test_audio = new_feature

        if input_details["dtype"] == np.uint8:
            input_scale, input_zero_point = input_details["quantization"]
            test_audio = test_audio / input_scale + input_zero_point
            test_audio = np.around(test_audio)

        test_audio = test_audio.astype(input_details["dtype"])
        interpreter.set_tensor(input_details["index"], [test_audio])
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]

        output_type = output_details["dtype"]
        if output_type == np.uint8:
            output_scale, output_zero_point = output_details["quantization"]
            output = output_scale * (output.astype(np.float32) - output_zero_point)
        out_label = ""
        if np.argmax(output) == 0:
            out_label = "Alarm"
        elif np.argmax(output) == 1:
            out_label = "Water"
        else:
            out_label = "Other"
        print(f"output:{i} / uint8 tflite-> {output}: keras-> {pred[0]}, label: {out_label}")


In [6]:
feature_folder = "features_new"
audio_folder = "data/balanced/inference"

audio_file = os.path.join(audio_folder, "Alarm.wav")
keras_model_file = os.path.join("weights", "mfcc_512_93%_model_16k_1.3.weights.h5")
tflite_model_file = os.path.join("weights", "mfcc_512_93%_model_16k_1.3.weights.tflite")


same_training = True
trim_flag = True

print("\n\n\n====================== Convert keras to qt tflite model ======================\n\n\n")
# """ # Do convert model from keras to quantized tflite """
# convert_keras2qttflite(feature_folder, keras_model_file, tflite_model_file)

print("\n\n\n====================== Compare keras to qt tflite model ======================\n\n\n")
""" # # Compare model with origin """




====================== Convert keras to qt tflite model ======================






====================== Compare keras to qt tflite model ======================





' # # Compare model with origin '

In [9]:
audio_file = os.path.join("data/realworld/audio_1.wav")

compare_model_kerasandtflite(audio_file, keras_model_file, tflite_model_file, trim_flag=trim_flag, same_training=same_training)

Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:0 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Alarm
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:10 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Alarm
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:20 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Alarm
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:30 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Alarm
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:40 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Alarm
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
output:50 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> 

In [ ]:
audio_file = os.path.join("data/balanced/test/Alarm/a (207).wav")

compare_model_kerasandtflite(audio_file, keras_model_file, tflite_model_file, trim_flag=trim_flag, same_training=same_training)

Original feature vector (257, 494)
After Split ....
Stft:  (257, 10)


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=10
  warnings.warn(


mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 13, 1)
mfcc after mean: (13, 257)
Stft:  (257, 10)
mfcc before mean: (257, 1

In [ ]:
audio_file = os.path.join("data/balanced/test/Water/water5_339.wav")

compare_model_kerasandtflite(audio_file, keras_model_file, tflite_model_file, trim_flag=trim_flag, same_training=same_training)

output:0 / uint8 tflite-> [0.         0.         0.99609375]: keras-> [0. 0. 1.], label: Alarm
output:10 / uint8 tflite-> [0.         0.         0.99609375]: keras-> [0. 0. 1.], label: Alarm
output:20 / uint8 tflite-> [0.         0.         0.99609375]: keras-> [0. 0. 1.], label: Alarm
output:30 / uint8 tflite-> [0.         0.         0.99609375]: keras-> [0. 0. 1.], label: Alarm
output:40 / uint8 tflite-> [0.         0.         0.99609375]: keras-> [6.8508787e-16 7.4933365e-17 1.0000000e+00], label: Alarm
output:50 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1.0000000e+00 1.6057102e-26 0.0000000e+00], label: Water
output:60 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [9.9999976e-01 2.9199893e-07 2.5884388e-12], label: Water
output:70 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1.0000000e+00 1.0506623e-20 0.0000000e+00], label: Water
output:80 / uint8 tflite-> [0.99609375 0.         0.        ]: keras-> [1. 0. 0.], label: Water
outpu